Stuff about the analysis

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from ExpressionTools import pyXset,  EsetPrep
from VizTools import volcano

/Users/coltongarelli/anaconda3/envs/ExprArch/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
R[write to console]: No methods found in package 'RSQLite' for request: 'dbListFields' when loading 'oligo'

R[write to console]: No methods found in package 'RSQLite' for request: 'dbListFields' when loading 'lumi'



In [2]:
prep = EsetPrep.GeoPrep("GSE81071", "data/GSE81071")
prep.load_geo()


R[write to console]: Found 1 file(s)

R[write to console]: GSE81071_series_matrix.txt.gz

R[write to console]: Using locally cached version: data/GSE81071/GSE81071_series_matrix.txt.gz

R[write to console]: Parsed with column specification:
cols(
  .default = col_double(),
  ID_REF = col_character()
)

R[write to console]: See spec(...) for full column specifications.

R[write to console]: Annotation GPL not available, so will use submitter GPL instead

R[write to console]: Using locally cached version of GPL19983 found here:
data/GSE81071/GPL19983.soft 



In [3]:
prep.eset.exprs

array([[ 0.80587924,  2.4769318 ,  1.5747854 , ...,  0.44647994,
         0.49297896,  0.20446866],
       [ 2.0238402 ,  2.1612492 ,  2.3555453 , ...,  0.87956375,
         1.42581602,  0.97506221],
       [ 6.352953  ,  5.8851805 ,  6.045486  , ..., -0.04958195,
         0.54319216, -0.22685148],
       ...,
       [ 4.3521414 ,  4.0144424 ,  2.750379  , ...,  1.15331793,
         1.69636648,  1.03240267],
       [ 6.9767294 ,  7.2071524 ,  7.027711  , ...,  0.69596142,
         0.88793491,  0.99032909],
       [ 1.89187251,  2.43857383,  1.55783043, ...,  0.57382672,
        -0.06659769, -0.04406841]])

In [4]:
batch_corrected = pd.read_csv("data/GSE81071/GSE81071_batch-corrected_matrix.txt", sep='\t')
batch_corrected.pop('EntrezGeneID_at')
batch_corrected.pop('Expressed')
batch_corrected.pop('Description')
batch_corrected.set_index('Symbol', inplace=True)
prep.eset.exprs = batch_corrected
xset = pyXset.PyXset(prep.eset)

In [5]:
dle_filter = xset.search_pdata('DLE', 'DLE')
xset.update_groups(dle_filter)
normal_filter = xset.search_pdata('Normal', 'Normal')
xset.update_groups(normal_filter)
xset.groups

,GSM2142137,GSM2142138,GSM2142139,GSM2142140,GSM2142141,GSM2142142,GSM2142143,GSM2142144,GSM2142145,GSM2142146,...,GSM3999348,GSM3999349,GSM3999351,GSM3999352,GSM3999353,GSM3999355,GSM3999356,GSM3999357,GSM3999359,GSM3999360
DLE,False,False,False,False,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
Normal,True,True,True,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Let's try some QC now


In [6]:
from VizTools import bar, density, plot_pca
from AnalysisTools import pca
import seaborn as sns

Run the GSEA

In [7]:
from AnalysisTools import gsea

In [8]:
xset.pdata
for_gsea = xset.exprs.loc[:, dle_filter]

for_gsea = for_gsea.join(xset.exprs.loc[:, normal_filter], how='inner')
gene_translate = xset.translate_gene_ids([i.split('_at')[0] for i in for_gsea.index.to_list()], 'entrezgene', ['symbol'])

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-25582...done.
Finished.
680 input query terms found no hit:
	['100127889', '100127974', '100128338', '100128364', '100128374', '100128573', '100129473', '1001294
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


/Users/coltongarelli/anaconda3/envs/ExprArch/lib/python3.7/site-packages/biothings_client/base.py:143: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



In [9]:
# %tb
genes = gene_translate.copy()
genes.index = [i+'_at' for i in gene_translate.index]
print(genes)
gene_translate_trunk = genes.loc[:, 'symbol']
for_gsea = for_gsea.join(gene_translate_trunk, how='inner')
for_gsea.set_index('symbol', inplace=True)
print(for_gsea)
# GSE81071_gsea.run_gsea(for_gsea, classes)

                    _id     _score      symbol notfound
100009613_at  100009613  24.246126   LINC02584      NaN
100009676_at  100009676  24.248596  ZBTB11-AS1      NaN
10000_at          10000  23.455528        AKT3      NaN
10001_at          10001  24.247318        MED6      NaN
10002_at          10002  24.247318       NR2E3      NaN
...                 ...        ...         ...      ...
9993_at            9993  23.456963       DGCR2      NaN
9994_at            9994  24.248596    CASP8AP2      NaN
9997_at            9997  24.246126        SCO2      NaN
999_at              999  22.933994        CDH1      NaN
9_at                  9  24.248741        NAT1      NaN

[25582 rows x 4 columns]
            GSM2142144  GSM2142145  GSM2142146  GSM2142147  GSM2142148  \
symbol                                                                   
LINC02584     1.227199    3.141701    1.534635    1.496193    1.042947   
ZBTB11-AS1    2.452342    3.303168    2.719499    2.344242    2.695080   
AKT3  

In [10]:
from AnalysisTools import gsea
GSE81071_gsea = gsea.GSEAnalysis(".", '/GSE81071/gsea')
classes = GSE81071_gsea.construct_classes('DLE', 'Healthy', len([i for i in dle_filter.to_list() if i]), len([i for i in normal_filter.to_list() if i]))

GSE81071_gsea.run_gsea(for_gsea, classes)

2020-05-28 13:56:19,024 Warning: dropping duplicated gene names, only keep the first values
2020-05-28 13:56:19,039 Warning: Input data contains NA, filled NA with 0
